Cargamos el dataframe del subproblema de ruteo

In [1]:
import pandas as pd

df_cw = pd.read_csv('resultados_CW_v2.csv')
print(df_cw.head())

   tienda                                              rutas  \
0       1  [[322, 352, 382, 412, 442, 472, 473, 443, 413,...   
1       2  [[1439, 1438, 1468, 1498, 1528, 1558, 1559, 15...   
2       3  [[1371, 1340, 1310, 1311, 1281, 1251, 1252, 12...   
3       4  [[1680, 1710, 1740, 1770, 1800, 1830, 1860, 18...   
4       5  [[1660, 1630, 1601, 1600, 1570, 1540, 1541, 15...   

                                               carga  \
0           [18267016, 71162396, 79738216, 79579833]   
1                                         [73796021]   
2                     [53177483, 65776720, 79320956]   
3           [28498795, 59700901, 79287419, 78771963]   
4  [45885567, 111424091, 126920108, 139503929, 13...   

                            distancia  carga_total  n_camiones_utilizados  \
0         [12.0, 42.73, 48.49, 48.63]    248747461                      4   
1                             [38.06]     73796021                      1   
2               [28.49, 33.72, 43.48]    198275

Separamos las distancias recorridas por cada vehículo en cada tienda

In [2]:
import ast

# Limpiar y transformar la columna 'distancia' (que está en formato string de lista)
df_cw['distancia'] = df_cw['distancia'].apply(ast.literal_eval)

In [3]:
distancias = []
tiendas = []
vehiculos = []

# Recorremos cada fila y extraemos los elementos de la lista de distancias
for i, row in df_cw.iterrows():
    tienda = row['tienda']
    dist_list = row['distancia']  # ya es lista
    for j, dist in enumerate(dist_list):
        distancias.append(float(dist))
        tiendas.append(tienda)
        vehiculos.append(j)  # índice como identificador del vehículo

# Creamos el DataFrame
df_distancias = pd.DataFrame({
    'tienda': tiendas,
    'vehiculo': vehiculos,
    'distancia': distancias
})

print(df_distancias.head())

   tienda  vehiculo  distancia
0       1         0      12.00
1       1         1      42.73
2       1         2      48.49
3       1         3      48.63
4       2         0      38.06


In [4]:
df_distancias_ordenado = df_distancias.sort_values(by='distancia').reset_index(drop=True)

Con las distancias, las agrupamos en clusters los cuales serán nuestros grupos k

In [5]:
import numpy as np
from sklearn.cluster import KMeans

# Utilizamos la columna 'distancia' del DataFrame ordenado
dist_array = df_distancias_ordenado['distancia'].values.reshape(-1, 1)

# Clustering con KMeans sobre estas distancias
K = 3
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
cluster_labels = kmeans.fit_predict(dist_array)

# Ordenamos los centroides y reasignamos etiquetas para que el cluster 0 tenga las distancias más bajas
centroids = kmeans.cluster_centers_.flatten()
sorted_indices = np.argsort(centroids)
label_map = {original: new for new, original in enumerate(sorted_indices)}

# Aplicamos el nuevo etiquetado
df_distancias_ordenado['cluster'] = cluster_labels
df_distancias_ordenado['cluster_ordenado'] = df_distancias_ordenado['cluster'].map(label_map)

print(df_distancias_ordenado.head())


   tienda  vehiculo  distancia  cluster  cluster_ordenado
0      13         0      10.65        0                 0
1       1         0      12.00        0                 0
2       4         0      14.83        0                 0
3       8         0      17.66        0                 0
4      12         0      25.24        0                 0


Y sacamos los centroides que utilizaremos para calcular los costos promedios

In [6]:
centroides_ordenados = df_distancias_ordenado.groupby('cluster_ordenado')['distancia'].mean().reset_index()
centroides_ordenados.columns = ['cluster_ordenado', 'centroide']

print(centroides_ordenados.head())

   cluster_ordenado  centroide
0                 0  27.057647
1                 1  44.859255
2                 2  72.690322


Cambiamos el formato de las rutas para utilizar los valores dentro

In [7]:
import ast

# Limpiar y transformar la columna 'distancia' (que está en formato string de lista)
df_cw['rutas'] = df_cw['rutas'].apply(ast.literal_eval)

In [8]:
print(df_cw.head())

   tienda                                              rutas  \
0       1  [[322, 352, 382, 412, 442, 472, 473, 443, 413,...   
1       2  [[1439, 1438, 1468, 1498, 1528, 1558, 1559, 15...   
2       3  [[1371, 1340, 1310, 1311, 1281, 1251, 1252, 12...   
3       4  [[1680, 1710, 1740, 1770, 1800, 1830, 1860, 18...   
4       5  [[1660, 1630, 1601, 1600, 1570, 1540, 1541, 15...   

                                               carga  \
0           [18267016, 71162396, 79738216, 79579833]   
1                                         [73796021]   
2                     [53177483, 65776720, 79320956]   
3           [28498795, 59700901, 79287419, 78771963]   
4  [45885567, 111424091, 126920108, 139503929, 13...   

                            distancia  carga_total  n_camiones_utilizados  \
0         [12.0, 42.73, 48.49, 48.63]    248747461                      4   
1                             [38.06]     73796021                      1   
2               [28.49, 33.72, 43.48]    198275

Calculamos los clientes que estarán en cada ruta, dado por el largo de la lista de la ruta

In [9]:
n_clientes_ruta = []

for _, row in df_distancias_ordenado.iterrows():
    tienda = row['tienda']
    vehiculo_idx = int(row['vehiculo'])  # forzamos entero

    rutas_lista = df_cw.loc[df_cw['tienda'] == tienda, 'rutas'].iloc[0]
    n_clientes = len(rutas_lista[vehiculo_idx])
    n_clientes_ruta.append(n_clientes)

# Añadir la columna
df_distancias_ordenado['n_clientes_ruta'] = n_clientes_ruta

In [10]:
print(df_distancias_ordenado.head())

   tienda  vehiculo  distancia  cluster  cluster_ordenado  n_clientes_ruta
0      13         0      10.65        0                 0               10
1       1         0      12.00        0                 0               13
2       4         0      14.83        0                 0               15
3       8         0      17.66        0                 0               17
4      12         0      25.24        0                 0               25


In [11]:
nk_por_cluster = df_distancias_ordenado.groupby('cluster_ordenado')['n_clientes_ruta'].sum().reset_index()
nk_por_cluster.columns = ['cluster_ordenado', 'n_k (total clientes)']

print(nk_por_cluster.head())

   cluster_ordenado  n_k (total clientes)
0                 0                   436
1                 1                  1333
2                 2                   430


Agrupamos los datos

In [12]:
nk_ck_cluster = pd.merge(nk_por_cluster, centroides_ordenados, on='cluster_ordenado')
nk_ck_cluster['c_k'] = nk_ck_cluster['centroide'] / nk_ck_cluster['n_k (total clientes)']

print(nk_ck_cluster.head())

   cluster_ordenado  n_k (total clientes)  centroide       c_k
0                 0                   436  27.057647  0.062059
1                 1                  1333  44.859255  0.033653
2                 2                   430  72.690322  0.169047


In [13]:
import numpy as np

# Definimos parámetros de la función de aceptación
beta = 0.0152
theta = 0.9
price_candidates = np.linspace(0, 55.9, 100)

# Calculamos precio óptimo para cada cluster
pks = []
uks = []

for _, row in nk_ck_cluster.iterrows():
    n_k = row['n_k (total clientes)']
    C_k = row['c_k']
    A_k = -beta * price_candidates + theta
    U_k = n_k * A_k * (price_candidates - C_k)
    idx_opt = np.argmax(U_k)
    pks.append(price_candidates[idx_opt])
    uks.append(U_k[idx_opt])

# Añadimos los resultados al DataFrame
nk_ck_cluster['p_k (precio óptimo)'] = pks
nk_ck_cluster['U_k (utilidad óptima)'] = uks

print(nk_ck_cluster.head())


   cluster_ordenado  n_k (total clientes)  centroide       c_k  \
0                 0                   436  27.057647  0.062059   
1                 1                  1333  44.859255  0.033653   
2                 2                   430  72.690322  0.169047   

   p_k (precio óptimo)  U_k (utilidad óptima)  
0            29.361616            5795.883068  
1            29.361616           17737.161501  
2            29.926263            5695.588972  


In [14]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.9 MB/s eta 0:00:00


In [15]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpInteger, LpContinuous

# Creamos el problema
model = LpProblem("Maximizar_Utilidad_Por_Cluster", LpMaximize)

# Parámetros
beta = 0.0152
theta = 0.9
price_candidates = np.linspace(0, 55.9, 100)  # 100 precios posibles
price_indices = list(range(len(price_candidates)))

# Variables de decisión: una variable binaria para cada combinación (cluster, precio)
x = {
    (k, i): LpVariable(f"x_{k}_{i}", cat='Binary')
    for k in nk_ck_cluster.index
    for i in price_indices
}

# Variables auxiliares: precio seleccionado por cluster (para restricción de orden)
p_vars = {
    k: LpVariable(f"p_{k}", lowBound=0, cat=LpContinuous)
    for k in nk_ck_cluster.index
}

# Función objetivo
model += lpSum([
    row['n_k (total clientes)'] *
    ((-beta * price_candidates[i] + theta) *
     (price_candidates[i] - row['c_k']) * x[(k, i)])
    for k, row in nk_ck_cluster.iterrows()
    for i in price_indices
])

# Restricción 1: solo un precio puede ser seleccionado por cluster
for k in nk_ck_cluster.index:
    model += lpSum([x[(k, i)] for i in price_indices]) == 1

# Restricción 2: definir p_k como el precio efectivo elegido (para ordenamiento)
for k in nk_ck_cluster.index:
    model += p_vars[k] == lpSum([price_candidates[i] * x[(k, i)] for i in price_indices])

# Restricción 3: p_k >= c_k
for k, row in nk_ck_cluster.iterrows():
    model += p_vars[k] >= row['c_k']

# Restricción 4: p_{k+1} >= p_k (orden creciente por cluster ordenado)
cluster_indices = list(nk_ck_cluster.index)
for k1, k2 in zip(cluster_indices, cluster_indices[1:]):
    model += p_vars[k2] >= p_vars[k1]

# Resolver el modelo
model.solve()

# Extraer precios óptimos
optimal_prices = []
optimal_utils = []

for k in nk_ck_cluster.index:
    selected_i = [i for i in price_indices if x[(k, i)].value() == 1][0]
    p_opt = price_candidates[selected_i]
    n_k = nk_ck_cluster.loc[k, 'n_k (total clientes)']
    c_k = nk_ck_cluster.loc[k, 'c_k']
    A_k = -beta * p_opt + theta
    U_k = n_k * A_k * (p_opt - c_k)
    optimal_prices.append(p_opt)
    optimal_utils.append(U_k)

# Actualizar el DataFrame
nk_ck_cluster['p_k (entero óptimo)'] = optimal_prices
nk_ck_cluster['U_k (entero óptimo)'] = optimal_utils

print(nk_ck_cluster.head())

   cluster_ordenado  n_k (total clientes)  centroide       c_k  \
0                 0                   436  27.057647  0.062059   
1                 1                  1333  44.859255  0.033653   
2                 2                   430  72.690322  0.169047   

   p_k (precio óptimo)  U_k (utilidad óptima)  p_k (entero óptimo)  \
0            29.361616            5795.883068            29.361616   
1            29.361616           17737.161501            29.361616   
2            29.926263            5695.588972            29.926263   

   U_k (entero óptimo)  
0          5795.883068  
1         17737.161501  
2          5695.588972  


In [16]:
nk_ck_cluster.to_csv('nk_ck_cluster.csv', index=False)

In [17]:
utilidad_total = nk_ck_cluster['U_k (utilidad óptima)'].sum()
utilidad_total_e = nk_ck_cluster['U_k (entero óptimo)'].sum()


print(utilidad_total)
print(utilidad_total_e)

29228.633541136853
29228.633541136853
